In [1]:
import pandas as pd, numpy as np
from datetime import datetime
pd.set_option('max_colwidth',200)
# import yfinance as yf
import tushare as ts
ts.set_token('2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67')
import time, urllib
import glob
import os

In [2]:
def get_data(code,start,end):
    df = ts.pro_bar(ts_code=code, adj='qfq', start_date=start, end_date=end)
    return df

#获取当前交易日最新的股票代码和简称
def get_code():
    codes = pro.stock_basic(list_status='L').ts_code.values
    return codes


def download_tushare_stocks_data(start, end, ch_db_path, ticker_list):
    count = 1
    for ticker in ticker_list:
        if count%200==0:
            print("=======================Sleeping======================")
            time.sleep(60)
        else:
            if not os.path.exists(ch_db_path+ticker+".csv"):
                print("{} is new, start downloading now...".format(ticker))
                try:
                    data = get_data(ticker)
    #                 print(data)
                    data['trade_date'] = data['trade_date'].astype(str)
                    data['trade_date'] = data['trade_date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:] if len(x)!=10 else x)
                    data.sort_values("trade_date", inplace = True)
                    data.to_csv(ch_db_path+ticker+".csv", index = False)
                    print("{} data file created: {}".format(ticker, end))
                except Exception as e:
                    print(e)

            else:
                print("Already have data csv for {}".format(ticker))
                hist_data = pd.read_csv(ch_db_path+ticker+".csv")   
                hist_data['trade_date'] = hist_data['trade_date'].astype(str)
                hist_data['trade_date'] = hist_data['trade_date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:] if len(x)!=10 else x)
                hist_data.to_csv(ch_db_path+ticker+".csv", index = False)
                hist_data = pd.read_csv(ch_db_path+ticker+".csv")
                try:
                    hist_data_last_date = hist_data['trade_date'].values[-1]        
                    if today > hist_data_last_date:
                        print("Needs to update, start updating new data for {} now...".format(ticker))
                        update_start = hist_data_last_date
                        update_end = today
                        with eventlet.Timeout(60,False):
                            try:
                                new_data = get_data(ticker, update_start, update_end)
                                new_data['trade_date'] = new_data['trade_date'].astype(str)
                                new_data['trade_date'] = new_data['trade_date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:])
                                new_data.to_csv(ch_db_path+ticker+".csv", mode='a', header=False, index = False)
                                updated_duplicated_df = pd.read_csv(ch_db_path+ticker+".csv")
                                updated_df = updated_duplicated_df.drop_duplicates("trade_date")
                                updated_df.sort_values("trade_date", inplace = True)
                                updated_df.to_csv(ch_db_path+ticker+".csv", index = False)
                                print("New data updated till today for {}!".format(ticker))
                            except Exception as e:
                                print(e)
            #             print("Timed Out: Update Failed!")
                    else:
                        print("There's no new data to update for {}.".format(ticker))
                except Exception as e:
                    print(e)

        #     print("Data Download/Update for {} is Finished.".format(ticker))
            print("=======================Executed: {}=======================".format(count))
        count+=1
    print("【Updated Finished for today!】")


#设置token
token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
pro = ts.pro_api(token)
codes = get_code()
print("Public Company Numbers by today: ", len(codes))

today = str(datetime.now().date())
start = '20100101'
end = today
ch_db_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
ticker_list = codes

Public Company Numbers by today:  3754


In [ ]:
import eventlet
eventlet.monkey_patch()

download_tushare_stocks_data(start, end, ch_db_path, ticker_list)

Already have data csv for 000001.SZ
Needs to update, start updating new data for 000001.SZ now...


/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '2.6.2' or newer of 'numexpr' (version '2.6.1' currently installed).
  warnings.warn(msg, UserWarning)
/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


New data updated till today for 000001.SZ!
=======================Executed: 1=======================
Already have data csv for 000002.SZ
Needs to update, start updating new data for 000002.SZ now...
New data updated till today for 000002.SZ!
=======================Executed: 2=======================
Already have data csv for 000004.SZ
Needs to update, start updating new data for 000004.SZ now...
New data updated till today for 000004.SZ!
=======================Executed: 3=======================
Already have data csv for 000005.SZ
Needs to update, start updating new data for 000005.SZ now...
New data updated till today for 000005.SZ!
=======================Executed: 4=======================
Already have data csv for 000006.SZ
Needs to update, start updating new data for 000006.SZ now...
New data updated till today for 000006.SZ!
=======================Executed: 5=======================
Already have data csv for 000007.SZ
Needs to update, start updating new data for 000007.SZ now...
New d


KeyboardInterrupt

